![Ejemplo 2](images/Ej2_exp_trans.png)

In [272]:
from pyomo.environ import *
import pandas as pd

model = ConcreteModel()

Leemos los datos de las corrientes calientes

In [273]:
Hot_df = pd.read_excel('Datos/Ejercicio2.xlsx', 'Hot',skiprows = 1,index_col=0)
Hot_df

,0,1,2,3,4,5
ID,,,,,,
H1,0,0,60,70,90,60
H2,0,0,0,140,180,120


E igualmente de las frías.

In [274]:
Cold_df = pd.read_excel('Datos/Ejercicio2.xlsx', 'Cold',skiprows = 1,index_col=0)
Cold_df

,0,1,2,3,4,5
ID,,,,,,
C1,0,30,90,105,135,0
C2,0,0,0,0,117,78


In [275]:
hotst = Hot_df.index.values.tolist()
coldst = Cold_df.index.values.tolist()
s=hotst.copy()
s.insert(0,'S')
w=coldst.copy()
w.insert(0,'W')

model.i=Set(initialize=hotst)
model.j=Set(initialize=coldst)
model.s=Set(initialize=s)
model.S=Set(initialize='S')
model.w=Set(initialize=w)
model.W=Set(initialize='W')
model.k=Set(initialize=Hot_df.columns)


Podemos ver lo que hemos creado hasta ahora haciendo un model.pprint()

In [276]:
model.pprint()

7 Set Declarations
    S : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    1 :  {'S',}
    W : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    1 :  {'W',}
    i : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'H1', 'H2'}
    j : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'C1', 'C2'}
    k : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}
    s : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'S', 'H1', 'H2'}
    w : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :   

Creamos ahora las variables del modelo. Hay i potenciales aportes de calor por utilities calientes (Qs), uno en cada intervalo, al igual que de frías (Qw). De esta forma damos la oportunidad de que entren las utilities en cualquier nivel de temperatura. Creamos también i residuos (R), aunque sabemos que no habrá residuo del intervalo 0 ni tampoco del intervalo 9, lo cual implementaremos como dos restricciones diferentes.

In [277]:
Qs = model.Qs = Var(model.k,within = NonNegativeReals) 
Qw = model.Qw = Var(model.k,within = NonNegativeReals) 
R = model.R = Var(model.s,model.k,within = NonNegativeReals)
Q = model.Q = Var(model.s,model.w,model.k,within = NonNegativeReals)

Buscamos la minimización del vapor del intervalo 1 más la utility fría del intervalo 9.

In [278]:
model.util = Objective(expr = model.Qs[1] + model.Qw[5])

Constraints

In [279]:
nk = list(model.k)[1:] #Creamos una lista que contenga los índices sobre los que haremos el balance. En nuestro caso el balance lo aplicamos en todos los intervalos, excepto el intervalo 0. Es decir sobre 1-5, lo cual expresamos por [1:], es decir, desde 1 hasta el final de la lista.
model.C1 = ConstraintList()
for k in nk:
    for i in model.i:
        model.C1.add(
         R[i,k-1]+Hot_df.loc[i,k] == R[i,k]+sum(Q[i,j,k] for j in model.j)+sum(Q[i,w,k] for w in model.W)
         )
model.C2 = ConstraintList()
for k in nk:
    for s in model.S:
        model.C2.add(
         R[s,k-1]+Qs[k] == R[s,k]+sum(Q[s,j,k] for j in model.j)
         )        

model.C3 = ConstraintList()
for k in nk:
    for j in model.j:
        model.C3.add(
         sum(Q[i,j,k] for i in model.i)+ sum(Q[i,j,k] for i in model.S) == Cold_df.loc[j,k]
        )

model.C4 = ConstraintList()
for k in nk:
        model.C4.add(
         sum(Q[i,'W',k] for i in model.i)  == Qw[k]
        )


model.C5 = ConstraintList()
for s in model.s:
    model.C5.add(
         R[s,0]==0
        )

model.C6 = ConstraintList()
for s in model.s:
    model.C6.add(
         R[s,5]==0
        )

nm = list(model.k)[0:1]+list(model.k)[2:]
model.C7 = ConstraintList()
for m in nm:
    model.C7.add(
         Qs[m]==0
        )

nn = list(model.k)[0:5]
model.C8 = ConstraintList()
for n in nn:
    model.C8.add(
         Qw[n]==0
        )

model.C9 = ConstraintList()
for k in model.k:
    model.C9.add(
        Q['S','W',k] ==0
    )

Resolvemos el modelo

In [280]:
results = SolverFactory('glpk').solve(model)
model.pprint()
results.write()

18 Set Declarations
    C1_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
    C2_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    C3_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
    C4_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    C5_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    C6_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    C7_index : Size=1, Index=N

In [281]:
Qss = value(model.Qs[1])
Qww = value(model.Qw[5])
print('Cold utility = {0:2.2f}, Hot utility = {1:2.2f}'.format(Qww, Qss))

Cold utility = 225.00, Hot utility = 60.00
